# Extending Auto-Sklearn with Catboost Component

In [1]:
from ConfigSpace.configuration_space import ConfigurationSpace
from ConfigSpace.hyperparameters import UniformFloatHyperparameter, \
    UniformIntegerHyperparameter, CategoricalHyperparameter

import sklearn.metrics
import autosklearn.regression
import autosklearn.pipeline.components.regression
from autosklearn.pipeline.components.base import AutoSklearnRegressionAlgorithm
from autosklearn.pipeline.constants import SPARSE, DENSE, \
    SIGNED_DATA, UNSIGNED_DATA, PREDICTIONS

from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

from catboost import CatBoostRegressor
print("autosklearn", autosklearn.__version__)

import sklearn.metrics
from sklearn.metrics import *


from platform import python_version
print(python_version())

autosklearn 0.10.0
3.7.7


/opt/conda/lib/python3.7/site-packages/pyparsing.py:3190: FutureWarning: Possible set intersection at position 3
  self.re = re.compile(self.reString)


## Generate data

In [2]:
X, y = load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y)

## Test catboost Regressor 

In [3]:
# cat_model = CatBoostRegressor()
#                               nan_mode='Min',eval_metric='RMSE',iterations= 1000,sampling_frequency='PerTree',
#                               leaf_estimation_method='Newton',grow_policy='SymmetricTree',penalties_coefficient=1,
#                               boosting_type='Plain',model_shrink_mode='Constant',feature_border_type='GreedyLogSum',
#                               l2_leaf_reg= 3,random_strength= 1,
#                               rsm= 1, boost_from_average=True,model_size_reg= 0.5,subsample=0.800000011920929,
#                               use_best_model= False,random_seed=0,depth=6,posterior_sampling= False,
#                               border_count=254,
#                               sparse_features_conflict_fraction= 0,leaf_estimation_backtracking='AnyImprovement',
#                               best_model_min_trees=1,model_shrink_rate=0,min_data_in_leaf=1,loss_function='RMSE',
#                               learning_rate=0.07300599664449692,score_function='Cosine',task_type='CPU',
#                               leaf_estimation_iterations=1,bootstrap_type='MVS',max_leaves=64, bagging_temperature=1)

# cat_model.fit(X_train, y_train, verbose=1)
# print(cat_model.get_all_params())
# cat_model.predict(X_test)

## Implement LGBcomponent

In [4]:
class CatBoostReg(AutoSklearnRegressionAlgorithm):
    def __init__(self,iterations, learning_rate, depth, l2_leaf_reg, bagging_temperature, random_state=None):
        self.iterations = iterations
        self.learning_rate = learning_rate
        self.depth=depth
        self.l2_leaf_reg=l2_leaf_reg
        self.bagging_temperature=bagging_temperature

        
        self.random_state = random_state
        self.estimator = None
        
    def fit(self, X, y):
        from catboost import CatBoostRegressor 
        self.estimator = CatBoostRegressor(iterations=self.iterations, learning_rate=self.learning_rate
                                           ,depth=self.depth, l2_leaf_reg=self.l2_leaf_reg
                                           ,bagging_temperature=self.bagging_temperature, 
                                            )
        self.estimator.fit(X, y)
        return self
    
    def predict(self, X):
        if self.estimator is None:
            raise NotImplementedError
        return self.estimator.predict(X)
    
    @staticmethod
    def get_properties(dataset_properties=None):
        return {'shortname': 'CatBoostReg',
                'name': 'CatBoos Regression',
                'handles_regression': True,
                'handles_classification': False,
                'handles_multiclass': False,
                'handles_multilabel': False,
                'handles_multioutput': True,
                'is_deterministic': True,
                'input': (SPARSE, DENSE, UNSIGNED_DATA, SIGNED_DATA),
                'output': (PREDICTIONS,)}

    @staticmethod
    def get_hyperparameter_search_space(dataset_properties=None):
        cs = ConfigurationSpace()
        iterations=UniformIntegerHyperparameter(
            name='iterations', lower=1, upper=3000, default_value=1000)
        learning_rate = UniformFloatHyperparameter(
            name='learning_rate', lower=0.000000001, upper=1,default_value=0.03) #log=True
        depth=UniformIntegerHyperparameter(
            name='depth', lower=1, upper=12, default_value=6)
        l2_leaf_reg=UniformFloatHyperparameter(
            name='l2_leaf_reg', lower=0.0, upper=5.0, default_value=3.0) #log=True
        bagging_temperature=UniformFloatHyperparameter(
            name='bagging_temperature', lower=0.0, upper=2.0, default_value=1.0) #log=True
        
        cs.add_hyperparameters([iterations,learning_rate,depth,l2_leaf_reg,bagging_temperature])
        return cs
    
# Add XGB component to auto-sklearn.
autosklearn.pipeline.components.regression.add_regressor(CatBoostReg)
cs = CatBoostReg.get_hyperparameter_search_space()
print(cs)

Configuration space object:
  Hyperparameters:
    bagging_temperature, Type: UniformFloat, Range: [0.0, 2.0], Default: 1.0
    depth, Type: UniformInteger, Range: [1, 12], Default: 6
    iterations, Type: UniformInteger, Range: [1, 3000], Default: 1000
    l2_leaf_reg, Type: UniformFloat, Range: [0.0, 5.0], Default: 3.0
    learning_rate, Type: UniformFloat, Range: [1e-09, 1.0], Default: 0.03



## Build and Fit the model using the created XBG component

In [5]:
automl = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=30 
    ,per_run_time_limit=10
    ,ensemble_size=1
    ,include_estimators=['CatBoostReg'] 
    ,initial_configurations_via_metalearning=0,
)
automl.fit(X_train, y_train)

0:	learn: 77.9259201	total: 46.4ms	remaining: 46.4s
1:	learn: 77.0422161	total: 47.3ms	remaining: 23.6s
2:	learn: 75.9702035	total: 47.9ms	remaining: 15.9s
3:	learn: 75.3176141	total: 48.5ms	remaining: 12.1s
4:	learn: 74.3753318	total: 49.1ms	remaining: 9.78s
5:	learn: 73.6192168	total: 49.7ms	remaining: 8.24s
6:	learn: 72.9135978	total: 50.5ms	remaining: 7.16s
7:	learn: 72.2264946	total: 51.9ms	remaining: 6.43s
8:	learn: 71.4592272	total: 52.6ms	remaining: 5.8s
9:	learn: 70.5557453	total: 53.2ms	remaining: 5.27s
10:	learn: 69.7967662	total: 53.8ms	remaining: 4.84s
11:	learn: 69.1332828	total: 55.1ms	remaining: 4.54s
12:	learn: 68.3810071	total: 55.5ms	remaining: 4.22s
13:	learn: 67.6863696	total: 56.7ms	remaining: 4s
14:	learn: 67.0735517	total: 57.6ms	remaining: 3.78s
15:	learn: 66.3979076	total: 58.4ms	remaining: 3.59s
16:	learn: 65.7248097	total: 59.1ms	remaining: 3.42s
17:	learn: 65.0565843	total: 59.5ms	remaining: 3.25s
18:	learn: 64.4405177	total: 60ms	remaining: 3.1s
19:	learn:

156:	learn: 35.4225049	total: 151ms	remaining: 811ms
157:	learn: 35.3347041	total: 152ms	remaining: 810ms
158:	learn: 35.2475025	total: 153ms	remaining: 807ms
159:	learn: 35.1232873	total: 153ms	remaining: 804ms
160:	learn: 35.0011972	total: 154ms	remaining: 803ms
161:	learn: 34.9007631	total: 155ms	remaining: 799ms
162:	learn: 34.8218731	total: 155ms	remaining: 797ms
163:	learn: 34.7510444	total: 156ms	remaining: 795ms
164:	learn: 34.6903103	total: 156ms	remaining: 792ms
165:	learn: 34.6201248	total: 157ms	remaining: 788ms
166:	learn: 34.5274744	total: 158ms	remaining: 786ms
167:	learn: 34.4462075	total: 158ms	remaining: 783ms
168:	learn: 34.3690287	total: 159ms	remaining: 780ms
169:	learn: 34.2509684	total: 159ms	remaining: 777ms
170:	learn: 34.1991952	total: 160ms	remaining: 774ms
171:	learn: 34.1665891	total: 160ms	remaining: 771ms
172:	learn: 34.0190135	total: 161ms	remaining: 769ms
173:	learn: 33.9360370	total: 161ms	remaining: 766ms
174:	learn: 33.8432599	total: 162ms	remaining:

311:	learn: 23.8327654	total: 258ms	remaining: 570ms
312:	learn: 23.8050591	total: 259ms	remaining: 569ms
313:	learn: 23.7264576	total: 260ms	remaining: 568ms
314:	learn: 23.6442131	total: 260ms	remaining: 566ms
315:	learn: 23.5661425	total: 261ms	remaining: 565ms
316:	learn: 23.4991758	total: 262ms	remaining: 565ms
317:	learn: 23.3983070	total: 263ms	remaining: 564ms
318:	learn: 23.3398128	total: 264ms	remaining: 564ms
319:	learn: 23.3302113	total: 265ms	remaining: 564ms
320:	learn: 23.2664983	total: 267ms	remaining: 564ms
321:	learn: 23.2319904	total: 268ms	remaining: 564ms
322:	learn: 23.1794434	total: 269ms	remaining: 563ms
323:	learn: 23.1626283	total: 270ms	remaining: 563ms
324:	learn: 23.1246475	total: 271ms	remaining: 563ms
325:	learn: 23.0325914	total: 272ms	remaining: 562ms
326:	learn: 23.0134990	total: 273ms	remaining: 562ms
327:	learn: 22.9680266	total: 274ms	remaining: 562ms
328:	learn: 22.9023564	total: 275ms	remaining: 562ms
329:	learn: 22.8853862	total: 276ms	remaining:

466:	learn: 16.4467108	total: 382ms	remaining: 436ms
467:	learn: 16.4013795	total: 383ms	remaining: 436ms
468:	learn: 16.3444208	total: 384ms	remaining: 435ms
469:	learn: 16.3249563	total: 384ms	remaining: 433ms
470:	learn: 16.3215399	total: 385ms	remaining: 432ms
471:	learn: 16.2570982	total: 385ms	remaining: 431ms
472:	learn: 16.2288865	total: 386ms	remaining: 430ms
473:	learn: 16.1809591	total: 387ms	remaining: 429ms
474:	learn: 16.1324741	total: 387ms	remaining: 428ms
475:	learn: 16.0584285	total: 388ms	remaining: 427ms
476:	learn: 16.0195902	total: 389ms	remaining: 426ms
477:	learn: 15.9821443	total: 389ms	remaining: 425ms
478:	learn: 15.9595242	total: 390ms	remaining: 424ms
479:	learn: 15.9327665	total: 390ms	remaining: 423ms
480:	learn: 15.9249667	total: 391ms	remaining: 422ms
481:	learn: 15.8875403	total: 391ms	remaining: 420ms
482:	learn: 15.8761099	total: 392ms	remaining: 420ms
483:	learn: 15.8056356	total: 393ms	remaining: 419ms
484:	learn: 15.7554947	total: 393ms	remaining:

621:	learn: 11.7500798	total: 503ms	remaining: 305ms
622:	learn: 11.7468081	total: 503ms	remaining: 305ms
623:	learn: 11.7287732	total: 504ms	remaining: 304ms
624:	learn: 11.6890530	total: 505ms	remaining: 303ms
625:	learn: 11.6710319	total: 505ms	remaining: 302ms
626:	learn: 11.6355417	total: 506ms	remaining: 301ms
627:	learn: 11.6259046	total: 507ms	remaining: 300ms
628:	learn: 11.5953130	total: 508ms	remaining: 299ms
629:	learn: 11.5770921	total: 509ms	remaining: 299ms
630:	learn: 11.5639915	total: 509ms	remaining: 298ms
631:	learn: 11.5326342	total: 510ms	remaining: 297ms
632:	learn: 11.4987735	total: 510ms	remaining: 296ms
633:	learn: 11.4869573	total: 511ms	remaining: 295ms
634:	learn: 11.4534207	total: 512ms	remaining: 294ms
635:	learn: 11.4498659	total: 513ms	remaining: 293ms
636:	learn: 11.4335214	total: 513ms	remaining: 292ms
637:	learn: 11.4307967	total: 514ms	remaining: 291ms
638:	learn: 11.4136295	total: 514ms	remaining: 291ms
639:	learn: 11.3965216	total: 515ms	remaining:

...

1220:	learn: 0.0000000	total: 739ms	remaining: 193ms
1221:	learn: 0.0000000	total: 740ms	remaining: 192ms
1222:	learn: 0.0000000	total: 740ms	remaining: 192ms
1223:	learn: 0.0000000	total: 741ms	remaining: 191ms
1224:	learn: 0.0000000	total: 741ms	remaining: 191ms
1225:	learn: 0.0000000	total: 742ms	remaining: 190ms
1226:	learn: 0.0000000	total: 742ms	remaining: 189ms
1227:	learn: 0.0000000	total: 743ms	remaining: 189ms
1228:	learn: 0.0000000	total: 743ms	remaining: 188ms
1229:	learn: 0.0000000	total: 744ms	remaining: 187ms
1230:	learn: 0.0000000	total: 744ms	remaining: 187ms
1231:	learn: 0.0000000	total: 744ms	remaining: 186ms
1232:	learn: 0.0000000	total: 745ms	remaining: 185ms
1233:	learn: 0.0000000	total: 745ms	remaining: 185ms
1234:	learn: 0.0000000	total: 746ms	remaining: 184ms
1235:	learn: 0.0000000	total: 747ms	remaining: 184ms
1236:	learn: 0.0000000	total: 747ms	remaining: 183ms
1237:	learn: 0.0000000	total: 748ms	remaining: 182ms
1238:	learn: 0.0000000	total: 748ms	remaining:

1375:	learn: 0.0000000	total: 839ms	remaining: 100ms
1376:	learn: 0.0000000	total: 840ms	remaining: 99.4ms
1377:	learn: 0.0000000	total: 840ms	remaining: 98.8ms
1378:	learn: 0.0000000	total: 841ms	remaining: 98.2ms
1379:	learn: 0.0000000	total: 842ms	remaining: 97.6ms
1380:	learn: 0.0000000	total: 842ms	remaining: 97ms
1381:	learn: 0.0000000	total: 843ms	remaining: 96.3ms
1382:	learn: 0.0000000	total: 843ms	remaining: 95.7ms
1383:	learn: 0.0000000	total: 844ms	remaining: 95.1ms
1384:	learn: 0.0000000	total: 845ms	remaining: 94.5ms
1385:	learn: 0.0000000	total: 845ms	remaining: 93.9ms
1386:	learn: 0.0000000	total: 845ms	remaining: 93.2ms
1387:	learn: 0.0000000	total: 846ms	remaining: 92.6ms
1388:	learn: 0.0000000	total: 846ms	remaining: 92ms
1389:	learn: 0.0000000	total: 847ms	remaining: 91.4ms
1390:	learn: 0.0000000	total: 847ms	remaining: 90.7ms
1391:	learn: 0.0000000	total: 848ms	remaining: 90.1ms
1392:	learn: 0.0000000	total: 848ms	remaining: 89.5ms
1393:	learn: 0.0000000	total: 849

1528:	learn: 0.0000000	total: 943ms	remaining: 6.79ms
1529:	learn: 0.0000000	total: 944ms	remaining: 6.17ms
1530:	learn: 0.0000000	total: 944ms	remaining: 5.55ms
1531:	learn: 0.0000000	total: 945ms	remaining: 4.93ms
1532:	learn: 0.0000000	total: 945ms	remaining: 4.32ms
1533:	learn: 0.0000000	total: 946ms	remaining: 3.7ms
1534:	learn: 0.0000000	total: 947ms	remaining: 3.08ms
1535:	learn: 0.0000000	total: 948ms	remaining: 2.47ms
1536:	learn: 0.0000000	total: 949ms	remaining: 1.85ms
1537:	learn: 0.0000000	total: 950ms	remaining: 1.23ms
1538:	learn: 0.0000000	total: 950ms	remaining: 617us
1539:	learn: 0.0000000	total: 951ms	remaining: 0us
0:	learn: 68.4550429	total: 1.59s	remaining: 6m 15s


/opt/conda/lib/python3.7/site-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


AutoSklearnRegressor(dask_client=None,
                     delete_output_folder_after_terminate=True,
                     delete_tmp_folder_after_terminate=True,
                     disable_evaluator_output=False, ensemble_memory_limit=1024,
                     ensemble_nbest=50, ensemble_size=1,
                     exclude_estimators=None, exclude_preprocessors=None,
                     get_smac_object_callback=None,
                     include_estimators=['CatBoostReg'],
                     include_preprocessors=None,
                     initial_configurations_via_metalearning=0,
                     logging_config=None, max_models_on_disc=50,
                     metadata_directory=None, metric=None, ml_memory_limit=3072,
                     n_jobs=None, output_folder=None, per_run_time_limit=10,
                     resampling_strategy='holdout',
                     resampling_strategy_arguments=None, seed=1,
                     smac_scenario_args=None, time_left_for_th

## Print prediction 



In [6]:
y_pred = automl.predict(X_test)
y_pred

array([115.88532257, 102.43771362, 146.94703674, 123.00325012,
       171.73272705, 127.50030518, 298.24636841, 162.59970093,
       120.60940552, 213.66007996, 158.94007874, 213.52455139,
       178.00543213, 153.78379822, 135.73739624, 139.93814087,
        85.73117065,  95.13109589, 199.30197144, 130.81280518,
       147.58810425,  94.08468628, 111.38576508, 241.45471191,
       149.70172119, 152.42965698, 150.99674988, 126.30281067,
        99.4546051 ,  58.54019165, 253.59318542, 213.98739624,
       148.55134583,  98.8616333 ,  95.03408813, 108.47148132,
        82.99738312, 108.14699554,  65.04945374, 177.78263855,
       164.4566803 , 248.26361084, 187.49447632,  81.42883301,
       179.6756134 , 109.43301392, 207.61729431, 209.29335022,
       284.25021362, 126.05480194, 258.5741272 , 139.0579834 ,
       114.34571838, 185.49711609, 219.11630249, 137.8908844 ,
       147.07814026, 184.65287781, 136.98013306, 179.87794495,
       129.5345459 , 161.92552185, 190.72573853, 102.70

## Print search results

In [7]:
print(automl.sprint_statistics())

auto-sklearn results:
  Dataset name: b27564dd491b92cd598ff926fb643611
  Metric: r2
  Best validation score: 0.404075
  Number of target algorithm runs: 7
  Number of successful target algorithm runs: 6
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 1
  Number of target algorithms that exceeded the memory limit: 0



## Print model paramerts

In [8]:
print("r2 score: ", sklearn.metrics.r2_score(y_pred, y_test))
print(automl.show_models())

r2 score:  -0.26434300529750954
[(1.000000, SimpleRegressionPipeline({'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'no_encoding', 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'minority_coalescer', 'data_preprocessing:numerical_transformer:imputation:strategy': 'mean', 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'none', 'feature_preprocessor:__choice__': 'select_rates_regression', 'regressor:__choice__': 'CatBoostReg', 'data_preprocessing:categorical_transformer:category_coalescence:minority_coalescer:minimum_fraction': 0.003539298244204388, 'feature_preprocessor:select_rates_regression:alpha': 0.16892293763551783, 'feature_preprocessor:select_rates_regression:mode': 'fpr', 'feature_preprocessor:select_rates_regression:score_func': 'f_regression', 'regressor:CatBoostReg:bagging_temperature': 1.0683109269797655, 'regressor:CatBoostReg:depth': 9, 'regressor:CatBoostReg:iterations': 1009, 'regressor:CatB